# DATA PREPROCESSING

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve


In [43]:
admissions = pd.read_csv('admissions_202208161605.csv')
patients = pd.read_csv('patients_202208161605.csv')
diagnoses = pd.read_csv('diagnoses_icd_202208161605.csv')
procedures=pd.read_csv('procedures_icd_202208161605.csv')
labevents=pd.read_csv('labevents_202208161605.csv')
cptevents=pd.read_csv('cptevents_202208161605.csv')
drgcodes=pd.read_csv('drgcodes_202208161605.csv')

In [44]:
print(drgcodes.isnull().sum())

row_id               0
subject_id           0
hadm_id              0
drg_type             0
drg_code             0
description         63
drg_severity     58923
drg_mortality    58923
dtype: int64


In [45]:
# admissions=admissions.dropna()
# patients=patients.dropna()
# diagnoses=diagnoses.dropna()

In [46]:
# admit_counts = admissions['subject_id'].value_counts()
# print(admit_counts.describe())
# print(admit_counts.value_counts().sort_index())
print(admissions.shape)
print(patients.shape)
total_rows = len(admissions)
unique_subjects = admissions['subject_id'].nunique()

print(f"Total rows in cohort: {total_rows}")
print(f"Unique subject_ids: {unique_subjects}")

# Check for duplicates
duplicate_subjects = admissions['subject_id'].duplicated().sum()
print(f"Number of duplicate subject_id entries: {duplicate_subjects}")


(58976, 19)
(46520, 8)
Total rows in cohort: 58976
Unique subject_ids: 46520
Number of duplicate subject_id entries: 12456


In [47]:
hf_icd9 = [
    '39891','40201','40211','40291','40401','40403','40411','40413','40491','40493',
    '4280','4281','42820','42821','42822','42823','42830','42831','42832',
    '42833','42840','42841','42842','42843','4289'
]


In [48]:
#drop row id
diagnoses = diagnoses.drop(columns=['row_id'])
admissions = admissions.drop(columns=['row_id'])
procedures = procedures.drop(columns=['row_id'])
patients = patients.drop(columns=['row_id'])
labevents = labevents.drop(columns=['row_id'])
cptevents = cptevents.drop(columns=['row_id'])
drgcodes = drgcodes.drop(columns=['row_id'])

hf_diagnoses = diagnoses[diagnoses['icd9_code'].astype(str).isin(hf_icd9)]
hf_admissions = admissions[admissions['hadm_id'].isin(hf_diagnoses['hadm_id'])].copy()

In [49]:
# 3. Merge with patient demographic data
hf_cohort = hf_admissions.merge(patients[['subject_id', 'dob', 'gender']], on='subject_id', how='left')
hf_cohort = hf_cohort.merge(hf_diagnoses, on=['subject_id', 'hadm_id'], how='left')

# Merge with lab events to get lab test results
hf_labevents = labevents[labevents['hadm_id'].isin(hf_cohort['hadm_id'])]
# Average lab value (for numeric values)
lab_avg = hf_labevents.groupby('hadm_id')['valuenum'].mean().reset_index(name='avg_lab_value')
hf_cohort = hf_cohort.merge(lab_avg[['hadm_id', 'avg_lab_value']], on='hadm_id', how='left')

# Merge with cpt codes to count the number of procedures
hf_cptevents = cptevents[cptevents['hadm_id'].isin(hf_cohort['hadm_id'])]
cpt_code_count = hf_cptevents.groupby('hadm_id')['cpt_cd'].nunique().reset_index(name='cpt_code_count')
hf_cohort = hf_cohort.merge(cpt_code_count[['hadm_id', 'cpt_code_count']], on='hadm_id', how='left')

# Merge with DRG codes for severity
hf_drgcodes = drgcodes[drgcodes['hadm_id'].isin(hf_cohort['hadm_id'])]
drg_severity = hf_drgcodes[['hadm_id', 'drg_severity']].drop_duplicates()
hf_cohort = hf_cohort.merge(drg_severity, on='hadm_id', how='left')



hf_cohort['ethnicity_group'] = hf_cohort['ethnicity'].apply(lambda x: 'Non-Caucasian' if x != 'Caucasian' else 'Caucasian')

# Check how many unique subject_ids vs total rows
total_rows = len(hf_cohort)
print(hf_cohort)
unique_subjects = hf_cohort['subject_id'].nunique()

print(f"Total rows in cohort: {total_rows}")
print(f"Unique subject_ids: {unique_subjects}")

# Check for duplicates
duplicate_subjects = hf_cohort['subject_id'].duplicated().sum()
print(f"Number of duplicate subject_id entries: {duplicate_subjects}")


       subject_id  hadm_id                admittime                dischtime  \
0              26   197661  2126-05-06 15:16:00.000  2126-05-13 15:00:00.000   
1              30   104557  2172-10-14 14:17:00.000  2172-10-19 14:37:00.000   
2              34   115799  2186-07-18 16:46:00.000  2186-07-20 16:00:00.000   
3              34   115799  2186-07-18 16:46:00.000  2186-07-20 16:00:00.000   
4              34   144319  2191-02-23 05:23:00.000  2191-02-25 20:20:00.000   
...           ...      ...                      ...                      ...   
38233       98753   185764  2172-02-11 21:26:00.000  2172-03-02 08:00:00.000   
38234       98755   184156  2167-05-17 12:36:00.000  2167-05-18 17:03:00.000   
38235       98755   184156  2167-05-17 12:36:00.000  2167-05-18 17:03:00.000   
38236       98755   184156  2167-05-17 12:36:00.000  2167-05-18 17:03:00.000   
38237       98755   184156  2167-05-17 12:36:00.000  2167-05-18 17:03:00.000   

                     deathtime admissio

In [50]:
hf_cohort['dob'] = pd.to_datetime(hf_cohort['dob'], errors='coerce')
hf_cohort['admittime'] = pd.to_datetime(hf_cohort['admittime'], errors='coerce')
hf_cohort['dischtime'] = pd.to_datetime(hf_cohort['dischtime'], errors='coerce')
hf_cohort['length_of_stay'] = (hf_cohort['dischtime'] - hf_cohort['admittime']).dt.days
hf_cohort = hf_cohort[hf_cohort['length_of_stay'] >= 0]  # Remove any negative values
# Add the admission type directly from the admissions table
hf_cohort['admission_type'] = hf_cohort['admission_type'].astype(str)
# Example: Categorize based on ICD9 codes
print(hf_cohort.isnull().sum())
hf_cohort['icd9_category'] = hf_cohort['icd9_code'].apply(lambda x: 'Cardiovascular' if x in hf_icd9 else 'Other')
hf_cohort['insurance_type'] = hf_cohort['insurance'].astype(str)
hf_cohort['admit_hour'] = hf_cohort['admittime'].dt.hour
hf_cohort['admit_day'] = hf_cohort['admittime'].dt.dayofweek
print(hf_cohort)


subject_id                  0
hadm_id                     0
admittime                   0
dischtime                   0
deathtime               33373
admission_type              0
admission_location          0
discharge_location          0
insurance                   0
language                 7824
religion                   81
marital_status            941
ethnicity                   0
edregtime               13732
edouttime               13732
diagnosis                   6
hospital_expire_flag        0
has_chartevents_data        0
dob                         0
gender                      0
seq_num                     0
icd9_code                   0
avg_lab_value             143
cpt_code_count           1683
drg_severity            21261
ethnicity_group             0
length_of_stay              0
dtype: int64
       subject_id  hadm_id           admittime           dischtime  \
0              26   197661 2126-05-06 15:16:00 2126-05-13 15:00:00   
1              30   104557 2172-10-14

In [51]:
hf_cohort = hf_cohort.dropna(subset=['dob', 'admittime'])
# Remove rows where dob is after admittime (invalid)
hf_cohort = hf_cohort[hf_cohort['dob'] < hf_cohort['admittime']]

# Also remove extremely old people (e.g., born before 1850)
hf_cohort = hf_cohort[hf_cohort['dob'] >= pd.Timestamp('1850-01-01')]

In [52]:
hf_cohort['readmitted_within_30_days'] = 0
hf_cohort = hf_cohort.sort_values(by=['subject_id']).reset_index(drop=True)


for i in range(1, len(hf_cohort)):
    curr = hf_cohort.iloc[i]
    prev = hf_cohort.iloc[i - 1]
    if (curr['subject_id'] == prev['subject_id'] and
        (curr['admittime'] - prev['dischtime']).days <= 30 and
        (curr['admittime'] - prev['dischtime']).days > 0):
        hf_cohort.at[hf_cohort.index[i - 1], 'readmitted_within_30_days'] = 1


In [53]:
print(hf_cohort)

       subject_id  hadm_id           admittime           dischtime  \
0               3   145834 2101-10-20 19:08:00 2101-10-31 13:58:00   
1               9   150750 2149-11-09 13:06:00 2149-11-14 10:15:00   
2              21   109451 2134-09-11 12:17:00 2134-09-24 16:15:00   
3              21   109451 2134-09-11 12:17:00 2134-09-24 16:15:00   
4              26   197661 2126-05-06 15:16:00 2126-05-13 15:00:00   
...           ...      ...                 ...                 ...   
36386       99991   151118 2184-12-24 08:30:00 2185-01-05 12:15:00   
36387       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36388       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36389       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36390       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   

                     deathtime admission_type         admission_location  \
0                          NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1      

In [54]:
# age_years = (hf_cohort['admittime'] - hf_cohort['dob']).dt.days / 365.25
# hf_cohort = hf_cohort[age_years < 120]  # optional upper limit
# hf_cohort['age'] = age_years.astype(int)


print("DOB range:", hf_cohort['dob'].min(), "to", hf_cohort['dob'].max())
print("Admit range:", hf_cohort['admittime'].min(), "to", hf_cohort['admittime'].max())
# Remove future or too-far dates to prevent datetime64[ns] overflow
hf_cohort = hf_cohort[hf_cohort['admittime'] <= pd.Timestamp('2200-01-01')]
hf_cohort = hf_cohort[hf_cohort['dob'] <= pd.Timestamp('2200-01-01')]
hf_cohort['age'] = ((hf_cohort['admittime'].values.astype('int64') - 
                     hf_cohort['dob'].values.astype('int64')) 
                    / (1e9 * 60 * 60 * 24 * 365.25)).astype(int)



DOB range: 1850-02-09 00:00:00 to 2198-07-09 00:00:00
Admit range: 2100-06-28 19:29:00 to 2209-07-14 21:35:00


In [55]:
print(hf_cohort)

       subject_id  hadm_id           admittime           dischtime  \
0               3   145834 2101-10-20 19:08:00 2101-10-31 13:58:00   
1               9   150750 2149-11-09 13:06:00 2149-11-14 10:15:00   
2              21   109451 2134-09-11 12:17:00 2134-09-24 16:15:00   
3              21   109451 2134-09-11 12:17:00 2134-09-24 16:15:00   
4              26   197661 2126-05-06 15:16:00 2126-05-13 15:00:00   
...           ...      ...                 ...                 ...   
36386       99991   151118 2184-12-24 08:30:00 2185-01-05 12:15:00   
36387       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36388       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36389       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
36390       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   

                     deathtime admission_type         admission_location  \
0                          NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1      

In [56]:
negative_age = hf_cohort[hf_cohort['age'] <0]
print(negative_age)

       subject_id  hadm_id           admittime           dischtime deathtime  \
5              30   104557 2172-10-14 14:17:00 2172-10-19 14:37:00       NaN   
6              34   115799 2186-07-18 16:46:00 2186-07-20 16:00:00       NaN   
7              34   144319 2191-02-23 05:23:00 2191-02-25 20:20:00       NaN   
8              34   144319 2191-02-23 05:23:00 2191-02-25 20:20:00       NaN   
9              34   144319 2191-02-23 05:23:00 2191-02-25 20:20:00       NaN   
...           ...      ...                 ...                 ...       ...   
36120       99186   111707 2183-09-28 19:18:00 2183-10-08 16:15:00       NaN   
36231       99483   108597 2184-01-11 14:16:00 2184-01-19 16:15:00       NaN   
36232       99483   108597 2184-01-11 14:16:00 2184-01-19 16:15:00       NaN   
36233       99483   108597 2184-01-11 14:16:00 2184-01-19 16:15:00       NaN   
36234       99483   108597 2184-01-11 14:16:00 2184-01-19 16:15:00       NaN   

      admission_type         admission_

# Remove rows with negative age
hf_cohort = hf_cohort[hf_cohort['age'] >= 0]


In [57]:
# Remove rows with negative age
hf_cohort = hf_cohort[hf_cohort['age'] >= 0]
# hf_cohort.dropna()

In [58]:
hf_cohort.to_csv('final_hf_data.csv', index=False)

In [59]:
re = hf_cohort[hf_cohort['readmitted_within_30_days'] ==1]
print(re)

       subject_id  hadm_id           admittime           dischtime deathtime  \
25             68   170467 2173-12-15 16:16:00 2174-01-03 18:30:00       NaN   
178           356   135591 2132-08-05 00:09:00 2132-08-13 17:41:00       NaN   
269           518   143946 2109-06-27 04:45:00 2109-06-29 17:00:00       NaN   
278           530   151499 2119-03-13 16:33:00 2119-04-04 12:00:00       NaN   
321           631   168097 2124-01-10 21:59:00 2124-01-18 18:25:00       NaN   
...           ...      ...                 ...                 ...       ...   
36205       99439   145935 2179-03-18 11:27:00 2179-03-20 18:00:00       NaN   
36258       99538   194801 2180-02-13 17:46:00 2180-02-26 15:35:00       NaN   
36281       99562   129689 2183-02-02 15:13:00 2183-02-18 13:28:00       NaN   
36283       99562   129689 2183-02-02 15:13:00 2183-02-18 13:28:00       NaN   
36373       99982   151454 2156-11-28 11:56:00 2156-12-08 13:45:00       NaN   

      admission_type         admission_